In [1]:
# Import dependencies
import pandas as pd
import os
import kagglehub
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
import keras_tuner as kt
from sklearn.preprocessing import StandardScaler
from mrmr import mrmr_classif
from tensorflow.keras.layers import Dropout

# Read in our cleaned and processed dataset
df = pd.read_csv('Data/cleaned_alzheimers_dataset')

# Check the data
df.head()

,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,9.025679,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,7.151293,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,9.673574,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,8.392554,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,5.597238,...,6.045039,0,0,0.014691,0,0,1,1,0,0


We are going to Run a Random Forest Classifier combined with Grid Search to get an initial column importance rating

In [ ]:
# Create a copy of the dataframe
df_copy = df.copy()

# Step 2: Define features and target
X = df_copy.drop(columns='Diagnosis')
y = df_copy['Diagnosis']

# Step 3: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Initialize the model
rf = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [200, 300, 400],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

y_pred = best_rf.predict(X_test)
# Step 10: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Additional classification report
print(classification_report(y_test, y_pred))

# If you need to further evaluate feature importances
feature_importances = best_rf.feature_importances_
features = X.columns

# Create a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
Accuracy: 0.9465
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       277
           1       0.96      0.88      0.92       153

    accuracy                           0.95       430
   macro avg       0.95      0.93      0.94       430
weighted avg       0.95      0.95      0.95       430

                      Feature  Importance
23       FunctionalAssessment    0.202252
26                        ADL    0.176325
22                       MMSE    0.128243
24           MemoryComplaints    0.089677
25         BehavioralProblems    0.050370
8                 DietQuality    0.027584
21   CholesterolTriglycerides    0.027234
20             CholesterolHDL    0.026933
7            PhysicalActivity    0.026467
9                SleepQuality    0.026458
18           CholesterolTotal    0.026181
4                         BMI    0.025284
19             CholesterolLDL    0.024569
6    

In [ ]:
# Create my df
my_df = df.copy()

# MemoryComplaints, BehavioralProblems

# Declare columns to drop
columns_to_drop = ['FunctionalAssessment', 'ADL', 'MMSE']

# Grab only my columns to evaluate
my_df = my_df.drop(columns=columns_to_drop)

# Define features and target
X = my_df.drop(columns='Diagnosis')
y = my_df['Diagnosis']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Check dimensions
print(f"Training Data Shape: {X_train_scaled.shape}")
print(f"Test Data Shape: {X_test_scaled.shape}")

Training Data Shape: (1719, 29)
Test Data Shape: (430, 29)


In [30]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide activation function
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # First hidden layer
    nn_model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=25, max_value=200, step=25),
        activation=activation,
        input_dim=29
    ))

    # Hidden layers (1 to 4)
    for i in range(hp.Int('num_layers', 1, 4)):
        units = hp.Int(f'units_{i}', min_value=35, max_value=300, step=25)
        nn_model.add(tf.keras.layers.Dense(units, activation=activation))

    # Output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile with tunable optimizer
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    return nn_model


In [31]:
# Reset tuner if needed
import shutil
import os

# Define the tuner directory based on output
tuner_dir = os.path.join(".", "untitled_project")

# Delete the tuner directory
shutil.rmtree(tuner_dir, ignore_errors=True)

# Confirm deletion
print("Deleted tuner directory:", not os.path.exists(tuner_dir))

Deleted tuner directory: True


In [32]:
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define the Hyperband tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=70,  
    hyperband_iterations=8  
)

In [33]:
# Run the Keras Tuner search
tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

Trial 720 Complete [00h 00m 12s]
val_accuracy: 0.7325581312179565

Best val_accuracy So Far: 0.7488372325897217
Total elapsed time: 01h 01m 28s


In [34]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)


top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

{'activation': 'tanh', 'first_units': 25, 'num_layers': 1, 'units_0': 135, 'optimizer': 'sgd', 'units_1': 60, 'units_2': 210, 'units_3': 85, 'tuner/epochs': 24, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 100, 'num_layers': 4, 'units_0': 85, 'optimizer': 'adam', 'units_1': 135, 'units_2': 210, 'units_3': 185, 'tuner/epochs': 70, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 175, 'num_layers': 3, 'units_0': 185, 'optimizer': 'adam', 'units_1': 135, 'units_2': 135, 'units_3': 260, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 3, 'tuner/round': 0}
14/14 - 1s - loss: 0.5650 - accuracy: 0.7488 - 728ms/epoch - 52ms/step
Loss: 0.5649819374084473, Accuracy: 0.7488372325897217
14/14 - 1s - loss: 0.5568 - accuracy: 0.7488 - 673ms/epoch - 48ms/step
Loss: 0.5567501187324524, Accuracy: 0.7488372325897217
14/14 - 1s - loss: 0.5781 - accuracy: 0.7465 - 714ms/epoch - 51ms/s

These are the results of the tuner after running it with the correct column selection (i.e. dropping 'FunctionalAssessment', 'ADL', 'MMSE').

```
{'activation': 'tanh', 'first_units': 25, 'num_layers': 1, 'units_0': 135, 'optimizer': 'sgd', 'units_1': 60, 'units_2': 210, 'units_3': 85, 'tuner/epochs': 24, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 100, 'num_layers': 4, 'units_0': 85, 'optimizer': 'adam', 'units_1': 135, 'units_2': 210, 'units_3': 185, 'tuner/epochs': 70, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 175, 'num_layers': 3, 'units_0': 185, 'optimizer': 'adam', 'units_1': 135, 'units_2': 135, 'units_3': 260, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 3, 'tuner/round': 0}
14/14 - 1s - loss: 0.5650 - accuracy: 0.7488 - 728ms/epoch - 52ms/step
Loss: 0.5649819374084473, Accuracy: 0.7488372325897217
14/14 - 1s - loss: 0.5568 - accuracy: 0.7488 - 673ms/epoch - 48ms/step
Loss: 0.5567501187324524, Accuracy: 0.7488372325897217
14/14 - 1s - loss: 0.5781 - accuracy: 0.7465 - 714ms/epoch - 51ms/step
Loss: 0.5781014561653137, Accuracy: 0.7465116381645203
```

This was because these seemed to be more medically focused test/assements and our goal with this model is to try and predict Alzheimers disease in patients. These initial results didnt meet the requirments. Thus, we are going to try the tuner agian while using mRMR, which is a tool that selects the top features to trian the model on? __RILEY CLARIFY PLEAES__

In [ ]:
# Create my df
my_df = df.copy()

# MemoryComplaints, BehavioralProblems

# Declare columns to drop
columns_to_drop = ['DoctorInCharge', 'PatientID', 'FunctionalAssessment', 'ADL', 'MMSE']

# Grab only my columns to evaluate
my_df = my_df.drop(columns=columns_to_drop)

# Define features and target
X = my_df.drop(columns='Diagnosis')
y = my_df['Diagnosis']

# Perform mRMR feature selection to select top 15 features
selected_features = mrmr_classif(X=X, y=y, K=15)

# Keep only the selected features in X
X_selected = X[selected_features]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Check dimensions
print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape: {X_test.shape}")

100%|██████████| 15/15 [00:00<00:00, 48.68it/s]

Training Data Shape: (1719, 15)
Test Data Shape: (430, 15)


In [41]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide activation function
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # First hidden layer
    nn_model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=25, max_value=200, step=25),
        activation=activation,
        input_dim=15
    ))

    # Hidden layers (1 to 4)
    for i in range(hp.Int('num_layers', 1, 4)):
        units = hp.Int(f'units_{i}', min_value=35, max_value=300, step=25)
        nn_model.add(tf.keras.layers.Dense(units, activation=activation))

    # Output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile with tunable optimizer
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    return nn_model

In [42]:
# Reset tuner if needed
import shutil
import os

# Define the tuner directory based on output
tuner_dir = os.path.join(".", "untitled_project")

# Delete the tuner directory
shutil.rmtree(tuner_dir, ignore_errors=True)

# Confirm deletion
print("Deleted tuner directory:", not os.path.exists(tuner_dir))

Deleted tuner directory: True


In [43]:
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define the Hyperband tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=70,  
    hyperband_iterations=8  
)

In [44]:
# Run the Keras Tuner search
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping])

Trial 720 Complete [00h 00m 10s]
val_accuracy: 0.6465116143226624

Best val_accuracy So Far: 0.7465116381645203
Total elapsed time: 07h 03m 01s


In [47]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)


top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

{'activation': 'sigmoid', 'first_units': 125, 'num_layers': 1, 'units_0': 235, 'optimizer': 'adam', 'units_1': 135, 'units_2': 185, 'units_3': 35, 'tuner/epochs': 70, 'tuner/initial_epoch': 24, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0699'}
{'activation': 'sigmoid', 'first_units': 200, 'num_layers': 3, 'units_0': 260, 'optimizer': 'adam', 'units_1': 185, 'units_2': 185, 'units_3': 35, 'tuner/epochs': 70, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 150, 'num_layers': 1, 'units_0': 285, 'optimizer': 'adam', 'units_1': 285, 'units_2': 235, 'units_3': 60, 'tuner/epochs': 70, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
14/14 - 1s - loss: 0.5722 - accuracy: 0.7465 - 1s/epoch - 76ms/step
Loss: 0.5721879601478577, Accuracy: 0.7465116381645203
14/14 - 1s - loss: 0.5688 - accuracy: 0.7442 - 972ms/epoch - 69ms/step
Loss: 0.5687639117240906, Accuracy: 0.7441860437393188
14/14 - 1s - loss: 0.5704 - accura

These were the results of the mRMR with the top 15 features selected tuner:

```
{'activation': 'sigmoid', 'first_units': 125, 'num_layers': 1, 'units_0': 235, 'optimizer': 'adam', 'units_1': 135, 'units_2': 185, 'units_3': 35, 'tuner/epochs': 70, 'tuner/initial_epoch': 24, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0699'}
{'activation': 'sigmoid', 'first_units': 200, 'num_layers': 3, 'units_0': 260, 'optimizer': 'adam', 'units_1': 185, 'units_2': 185, 'units_3': 35, 'tuner/epochs': 70, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 150, 'num_layers': 1, 'units_0': 285, 'optimizer': 'adam', 'units_1': 285, 'units_2': 235, 'units_3': 60, 'tuner/epochs': 70, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
14/14 - 1s - loss: 0.5722 - accuracy: 0.7465 - 1s/epoch - 76ms/step
Loss: 0.5721879601478577, Accuracy: 0.7465116381645203
14/14 - 1s - loss: 0.5688 - accuracy: 0.7442 - 972ms/epoch - 69ms/step
Loss: 0.5687639117240906, Accuracy: 0.7441860437393188
14/14 - 1s - loss: 0.5704 - accuracy: 0.7372 - 642ms/epoch - 46ms/step
Loss: 0.5704463720321655, Accuracy: 0.7372093200683594
```

There doesn't seem to be a huge improvment. In fact, it seems like it did worse in some areas. Thus, we have also decided to test models with two of the columns, ADL and MMSE, to see how well our model could predict Alzheimers with one or both of these more medically relevent columns. Thus it is now time to build our models using the best hyperparameters and tweaking the feature sleection to get the most accurate and valuable models possible. 

In [2]:
# Create my df
my_df_1 = df.copy()

# Columns included: MemoryComplaints, BehavioralProblems

# Declare columns to drop
columns_to_drop = ['FunctionalAssessment', 'ADL', 'MMSE']

# Grab only my columns to evaluate
my_df_1 = my_df_1.drop(columns=columns_to_drop)

# Define features and target
X = my_df_1.drop(columns='Diagnosis')
y = my_df_1['Diagnosis']

# Perform mRMR feature selection to select top 15 features
selected_features = mrmr_classif(X=X, y=y, K=15)

# Keep only the selected features in X
X_selected = X[selected_features]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Check dimensions
print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape: {X_test.shape}")

100%|██████████| 15/15 [00:00<00:00, 34.97it/s]

Training Data Shape: (1719, 15)
Test Data Shape: (430, 15)


In [3]:
# Define the deep learning model with your specified architecture
nn_1_model = tf.keras.models.Sequential()

# Add the first layer (input layer)
nn_1_model.add(tf.keras.layers.Dense(units=125, activation="sigmoid", input_dim=15))
nn_1_model.add(Dropout(0.2))

# Add the subsequent hidden layers with the specified units
nn_1_model.add(tf.keras.layers.Dense(units=235, activation="sigmoid")) 
nn_1_model.add(Dropout(0.2))

nn_1_model.add(tf.keras.layers.Dense(units=135, activation="sigmoid")) 
nn_1_model.add(Dropout(0.2))

nn_1_model.add(tf.keras.layers.Dense(units=185, activation="sigmoid")) 
nn_1_model.add(Dropout(0.2))

nn_1_model.add(tf.keras.layers.Dense(units=35, activation="sigmoid")) 
nn_1_model.add(Dropout(0.2))

# Add the output layer (assuming binary classification with a sigmoid activation)
nn_1_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_1_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model (replace X_train_scaled and y_train with your actual data)
fit_model = nn_1_model.fit(X_train, y_train, epochs=100, initial_epoch=24)

# Evaluate the model using the test data (replace X_test_scaled and y_test with your actual test data)
model_loss, model_accuracy = nn_1_model.evaluate(X_test, y_test, verbose=2)

# Print the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



Epoch 25/100


54/54 [==============================] - 2s 5ms/step - loss: 0.6681 - accuracy: 0.6143
Epoch 26/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6560 - accuracy: 0.6446
Epoch 27/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6550 - accuracy: 0.6451
Epoch 28/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6549 - accuracy: 0.6469
Epoch 29/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6522 - accuracy: 0.6463
Epoch 30/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6502 - accuracy: 0.6469
Epoch 31/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6562 - accuracy: 0.6451
Epoch 32/100
54/54 [==============================] - 0s 6ms/step - loss: 0.6518 - accuracy: 0.6469
Epoch 33/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6524 - accuracy: 0.6469
Epoch 34/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6510 - accuracy: 0.6

In [4]:
# Create my df
my_df_2 = df.copy()

# Columns included: MemoryComplaints, BehavioralProblems, ADL
# Declare columns to drop
columns_to_drop = ['FunctionalAssessment', 'MMSE']

# Grab only my columns to evaluate
my_df_2 = my_df_2.drop(columns=columns_to_drop)

# Define features and target
X = my_df_2.drop(columns='Diagnosis')
y = my_df_2['Diagnosis']

# Perform mRMR feature selection to select top 15 features
selected_features = mrmr_classif(X=X, y=y, K=15)

# Keep only the selected features in X
X_selected = X[selected_features]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Check dimensions
print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape: {X_test.shape}")

100%|██████████| 15/15 [00:00<00:00, 30.19it/s]

Training Data Shape: (1719, 15)
Test Data Shape: (430, 15)


In [5]:
# Define the deep learning model with your specified architecture
nn_2_model = tf.keras.models.Sequential()

# Add the first layer (input layer)
nn_2_model.add(tf.keras.layers.Dense(units=125, activation="sigmoid", input_dim=15))
nn_2_model.add(Dropout(0.2))

# Add the subsequent hidden layers with the specified units
nn_2_model.add(tf.keras.layers.Dense(units=235, activation="sigmoid")) 
nn_2_model.add(Dropout(0.2))

nn_2_model.add(tf.keras.layers.Dense(units=135, activation="sigmoid")) 
nn_2_model.add(Dropout(0.2))

nn_2_model.add(tf.keras.layers.Dense(units=185, activation="sigmoid")) 
nn_2_model.add(Dropout(0.2))

nn_2_model.add(tf.keras.layers.Dense(units=35, activation="sigmoid")) 
nn_2_model.add(Dropout(0.2))

# Add the output layer (assuming binary classification with a sigmoid activation)
nn_2_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_2_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model (replace X_train_scaled and y_train with your actual data)
fit_model = nn_2_model.fit(X_train, y_train, epochs=100, initial_epoch=24)

# Evaluate the model using the test data (replace X_test_scaled and y_test with your actual test data)
model_loss, model_accuracy = nn_2_model.evaluate(X_test, y_test, verbose=2)

# Print the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 25/100
54/54 [==============================] - 1s 4ms/step - loss: 0.6661 - accuracy: 0.6277
Epoch 26/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6571 - accuracy: 0.6405
Epoch 27/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6538 - accuracy: 0.6440
Epoch 28/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6550 - accuracy: 0.6428
Epoch 29/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6556 - accuracy: 0.6463
Epoch 30/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6599 - accuracy: 0.6457
Epoch 31/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6507 - accuracy: 0.6469
Epoch 32/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6534 - accuracy: 0.6469
Epoch 33/100
54/54 [==============================] - 0s 5ms/step - loss: 0.6506 - accuracy: 0.6469
Epoch 34/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6513 - accuracy: 0.6469


In [6]:
# Create my df
my_df_3 = df.copy()

# Columns included: MemoryComplaints, BehavioralProblems, MMSE
# Declare columns to drop
columns_to_drop = ['FunctionalAssessment', 'ADL']

# Grab only my columns to evaluate
my_df_3 = my_df_3.drop(columns=columns_to_drop)

# Define features and target
X = my_df_3.drop(columns='Diagnosis')
y = my_df_3['Diagnosis']

# Perform mRMR feature selection to select top 15 features
selected_features = mrmr_classif(X=X, y=y, K=15)

# Keep only the selected features in X
X_selected = X[selected_features]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Check dimensions
print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape: {X_test.shape}")

100%|██████████| 15/15 [00:00<00:00, 36.89it/s]

Training Data Shape: (1719, 15)
Test Data Shape: (430, 15)


In [7]:
# Define the deep learning model with your specified architecture
nn_3_model = tf.keras.models.Sequential()

# Add the first layer (input layer)
nn_3_model.add(tf.keras.layers.Dense(units=125, activation="sigmoid", input_dim=15))
nn_3_model.add(Dropout(0.2))

# Add the subsequent hidden layers with the specified units
nn_3_model.add(tf.keras.layers.Dense(units=235, activation="sigmoid")) 
nn_3_model.add(Dropout(0.2))

nn_3_model.add(tf.keras.layers.Dense(units=135, activation="sigmoid")) 
nn_3_model.add(Dropout(0.2))

nn_3_model.add(tf.keras.layers.Dense(units=185, activation="sigmoid")) 
nn_3_model.add(Dropout(0.2))

nn_3_model.add(tf.keras.layers.Dense(units=35, activation="sigmoid")) 
nn_3_model.add(Dropout(0.2))

# Add the output layer (assuming binary classification with a sigmoid activation)
nn_3_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_3_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model (replace X_train_scaled and y_train with your actual data)
fit_model = nn_3_model.fit(X_train, y_train, epochs=100, initial_epoch=24)

# Evaluate the model using the test data (replace X_test_scaled and y_test with your actual test data)
model_loss, model_accuracy = nn_3_model.evaluate(X_test, y_test, verbose=2)

# Print the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 25/100
54/54 [==============================] - 1s 4ms/step - loss: 0.6819 - accuracy: 0.6161
Epoch 26/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6634 - accuracy: 0.6353
Epoch 27/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6641 - accuracy: 0.6364
Epoch 28/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6570 - accuracy: 0.6469
Epoch 29/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6567 - accuracy: 0.6422
Epoch 30/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6547 - accuracy: 0.6469
Epoch 31/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6540 - accuracy: 0.6469
Epoch 32/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6514 - accuracy: 0.6469
Epoch 33/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6524 - accuracy: 0.6469
Epoch 34/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6494 - accuracy: 0.6469


In [8]:
# Create my df
my_df_4 = df.copy()

# Columns included: MemoryComplaints, BehavioralProblems, MMSE, ADL
# Declare columns to drop
columns_to_drop = ['FunctionalAssessment']

# Grab only my columns to evaluate
my_df_4 = my_df_4.drop(columns=columns_to_drop)

# Define features and target
X = my_df_4.drop(columns='Diagnosis')
y = my_df_4['Diagnosis']

# Perform mRMR feature selection to select top 15 features
selected_features = mrmr_classif(X=X, y=y, K=15)

# Keep only the selected features in X
X_selected = X[selected_features]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Check dimensions
print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape: {X_test.shape}")

100%|██████████| 15/15 [00:00<00:00, 36.14it/s]

Training Data Shape: (1719, 15)
Test Data Shape: (430, 15)


In [9]:
# Define the deep learning model with your specified architecture
nn_4_model = tf.keras.models.Sequential()

# Add the first layer (input layer)
nn_4_model.add(tf.keras.layers.Dense(units=125, activation="sigmoid", input_dim=15))
nn_4_model.add(Dropout(0.2))

# Add the subsequent hidden layers with the specified units
nn_4_model.add(tf.keras.layers.Dense(units=235, activation="sigmoid")) 
nn_4_model.add(Dropout(0.2))

nn_4_model.add(tf.keras.layers.Dense(units=135, activation="sigmoid")) 
nn_4_model.add(Dropout(0.2))

nn_4_model.add(tf.keras.layers.Dense(units=185, activation="sigmoid")) 
nn_4_model.add(Dropout(0.2))

nn_4_model.add(tf.keras.layers.Dense(units=35, activation="sigmoid")) 
nn_4_model.add(Dropout(0.2))

# Add the output layer (assuming binary classification with a sigmoid activation)
nn_4_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_4_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model (replace X_train_scaled and y_train with your actual data)
fit_model = nn_4_model.fit(X_train, y_train, epochs=100, initial_epoch=24)

# Evaluate the model using the test data (replace X_test_scaled and y_test with your actual test data)
model_loss, model_accuracy = nn_4_model.evaluate(X_test, y_test, verbose=2)

# Print the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 25/100
54/54 [==============================] - 1s 4ms/step - loss: 0.6703 - accuracy: 0.6242
Epoch 26/100
54/54 [==============================] - 0s 5ms/step - loss: 0.6606 - accuracy: 0.6422
Epoch 27/100
54/54 [==============================] - 0s 4ms/step - loss: 0.6591 - accuracy: 0.6440
Epoch 28/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6502 - accuracy: 0.6469
Epoch 29/100
54/54 [==============================] - 0s 3ms/step - loss: 0.6325 - accuracy: 0.6475
Epoch 30/100
54/54 [==============================] - 0s 4ms/step - loss: 0.5913 - accuracy: 0.6946
Epoch 31/100
54/54 [==============================] - 0s 4ms/step - loss: 0.5718 - accuracy: 0.7132
Epoch 32/100
54/54 [==============================] - 0s 4ms/step - loss: 0.5560 - accuracy: 0.7341
Epoch 33/100
54/54 [==============================] - 0s 4ms/step - loss: 0.5615 - accuracy: 0.7184
Epoch 34/100
54/54 [==============================] - 0s 3ms/step - loss: 0.5498 - accuracy: 0.7179


Here are the results of each model:

Model with both ADL and MMSE removed:
```
14/14 - 0s - loss: 0.5879 - accuracy: 0.7000 - 225ms/epoch - 16ms/step
Loss: 0.5879404544830322, Accuracy: 0.699999988079071
```

Model with MMSE removed:
```
14/14 - 0s - loss: 0.5231 - accuracy: 0.7488 - 180ms/epoch - 13ms/step
Loss: 0.5231413841247559, Accuracy: 0.7488372325897217
```

Model with ADL removed:
```
14/14 - 0s - loss: 0.5844 - accuracy: 0.6953 - 297ms/epoch - 21ms/step
Loss: 0.5844082832336426, Accuracy: 0.695348858833313
```

Model with both ADL and MMSE included:
```
14/14 - 0s - loss: 0.4524 - accuracy: 0.7744 - 171ms/epoch - 12ms/step
Loss: 0.4524311423301697, Accuracy: 0.7744185924530029
```